## Tensoe flow + keras (mnist)

In [10]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
from sklearn.cross_validation import train_test_split
import tensorflow as tf
from keras.datasets import mnist
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.grid_search import GridSearchCV

/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:

seed = 7
np.random.seed(seed)

##### load data #####

(X_train, y_train), (X_test, y_test) = mnist.load_data()


11493376/11490434 [==============================] - 1s 0us/step


In [5]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

num_classes = 10



60000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
X_train.shape

(60000, 784)

In [11]:
def make_model(optimizer="adam", hidden_size=32):
    model = Sequential([
        Dense(hidden_size, input_shape=(784,),activation='relu'),
        Dense(10, activation='softmax'),
    ])
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model

clf = KerasClassifier(make_model)

param_grid = {'hidden_size': [32, 64, 128, 256]}

grid = GridSearchCV(clf, param_grid=param_grid, cv=5)

In [12]:
grid.fit(X_train, y_train)

best_param = grid.best_params_['hidden_size']

Epoch 1/1
12000/12000 [==============================] - 0s 24us/step
Epoch 1/1
12000/12000 [==============================] - 0s 22us/step
Epoch 1/1
12000/12000 [==============================] - 1s 42us/step
Epoch 1/1
12000/12000 [==============================] - 0s 27us/step
Epoch 1/1
12000/12000 [==============================] - 0s 25us/step
Epoch 1/1
12000/12000 [==============================] - 0s 26us/step
Epoch 1/1
12000/12000 [==============================] - 0s 30us/step
Epoch 1/1
12000/12000 [==============================] - 0s 29us/step
Epoch 1/1
12000/12000 [==============================] - 0s 31us/step
Epoch 1/1
12000/12000 [==============================] - 0s 39us/step
Epoch 1/1
12000/12000 [==============================] - 0s 34us/step
Epoch 1/1
12000/12000 [==============================] - 0s 34us/step
Epoch 1/1
12000/12000 [==============================] - 0s 36us/step
Epoch 1/1
12000/12000 [==============================] - 0s 35us/step
Epoch 1/1
12000/1200

In [13]:
### run vanilla model with best parameters #####
model_vanilla = Sequential([
    Dense(grid.best_params_['hidden_size'], input_shape=(784,), activation='relu'),
    Dense(10, activation='softmax'),
])
model_vanilla.compile("adam", "categorical_crossentropy", metrics=['accuracy'])


In [20]:
history_callback_vanilla = model_vanilla.fit(X_train, y_train, batch_size=128,
                             epochs=30, verbose=1, validation_split=.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/30
54000/54000 [==============================] - 4s 75us/step - loss: 0.3289 - acc: 0.9078 - val_loss: 0.1431 - val_acc: 0.9598
Epoch 2/30
54000/54000 [==============================] - 2s 36us/step - loss: 0.1431 - acc: 0.9584 - val_loss: 0.1047 - val_acc: 0.9732
Epoch 3/30
54000/54000 [==============================] - 2s 37us/step - loss: 0.0976 - acc: 0.9722 - val_loss: 0.0897 - val_acc: 0.9762
Epoch 4/30
54000/54000 [==============================] - 2s 43us/step - loss: 0.0718 - acc: 0.9790 - val_loss: 0.0802 - val_acc: 0.9775
Epoch 5/30
54000/54000 [==============================] - 2s 37us/step - loss: 0.0559 - acc: 0.9840 - val_loss: 0.0727 - val_acc: 0.9773
Epoch 6/30
54000/54000 [==============================] - 2s 45us/step - loss: 0.0434 - acc: 0.9878 - val_loss: 0.0763 - val_acc: 0.9780
Epoch 7/30
54000/54000 [==============================] - 2s 38us/step - loss: 0.0356 - acc: 0.9903 - val_loss: 0.0704 - val_acc:

In [14]:
## model selection for the dropout model #####
from keras.layers import Dropout

def drop_model(optimizer="adam", drop_param=0.5):
    model = Sequential([
        Dense(best_param, input_shape=(784,),activation='relu'),
        Dropout(drop_param),
        Dense(10, activation='softmax'),
    ])
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model

clf = KerasClassifier(drop_model)

param_grid = {'drop_param': [0.1, 0.3, 0.5, 0.6, 0.8]}

grid = GridSearchCV(clf, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)


Epoch 1/1
12000/12000 [==============================] - 1s 51us/step
Epoch 1/1
12000/12000 [==============================] - 1s 60us/step
Epoch 1/1
12000/12000 [==============================] - 1s 54us/step
Epoch 1/1
12000/12000 [==============================] - 1s 57us/step
Epoch 1/1
12000/12000 [==============================] - 1s 56us/step
Epoch 1/1
12000/12000 [==============================] - 1s 81us/step
Epoch 1/1
12000/12000 [==============================] - 1s 78us/step
Epoch 1/1
12000/12000 [==============================] - 1s 61us/step
Epoch 1/1
12000/12000 [==============================] - 1s 71us/step
Epoch 1/1
12000/12000 [==============================] - 1s 77us/step
Epoch 1/1
12000/12000 [==============================] - 1s 67us/step
Epoch 1/1
12000/12000 [==============================] - 1s 64us/step
Epoch 1/1
12000/12000 [==============================] - 1s 75us/step
Epoch 1/1
12000/12000 [==============================] - 1s 65us/step
Epoch 1/1
12000/1200

GridSearchCV(cv=5, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x12c9e2c50>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'drop_param': [0.1, 0.3, 0.5, 0.6, 0.8]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [15]:
### run dropout model with best parameters #####
model_dropout = Sequential([
    Dense(best_param, input_shape=(784,), activation='relu'),
    Dropout(grid.best_params_['drop_param']),
    Dense(10, activation='softmax'),
])
model_dropout.compile("adam", "categorical_crossentropy", metrics=['accuracy'])

In [24]:
history_callback_dropout = model_dropout.fit(X_train, y_train, batch_size=128,
                             epochs=30, verbose=1, validation_split=.1)


Train on 54000 samples, validate on 6000 samples
Epoch 1/30
54000/54000 [==============================] - 2s 44us/step - loss: 0.0131 - acc: 0.9955 - val_loss: 0.0736 - val_acc: 0.9835
Epoch 2/30
54000/54000 [==============================] - 2s 42us/step - loss: 0.0135 - acc: 0.9951 - val_loss: 0.0700 - val_acc: 0.9852
Epoch 3/30
54000/54000 [==============================] - 2s 38us/step - loss: 0.0121 - acc: 0.9963 - val_loss: 0.0677 - val_acc: 0.9855
Epoch 4/30
54000/54000 [==============================] - 2s 38us/step - loss: 0.0127 - acc: 0.9957 - val_loss: 0.0755 - val_acc: 0.9852
Epoch 5/30
54000/54000 [==============================] - 2s 38us/step - loss: 0.0110 - acc: 0.9962 - val_loss: 0.0690 - val_acc: 0.9847
Epoch 6/30
54000/54000 [==============================] - 2s 38us/step - loss: 0.0122 - acc: 0.9959 - val_loss: 0.0701 - val_acc: 0.9852
Epoch 7/30
54000/54000 [==============================] - 2s 38us/step - loss: 0.0105 - acc: 0.9965 - val_loss: 0.0735 - val_acc:

In [17]:
##### learning curve #####
def plot_history(logger):
    df = pd.DataFrame(logger.history)
    df[['acc', 'val_acc']].plot()
    plt.ylabel("accuracy")
    df[['loss', 'val_loss']].plot(linestyle='--', ax=plt.twinx())
    plt.ylabel("loss")

In [29]:
df = pd.DataFrame(history_callback_vanilla.history)
df[['acc', 'val_acc']].plot()
plt.ylabel("accuracy")
df[['loss', 'val_loss']].plot(linestyle='--', ax=plt.twinx())
plt.ylabel("loss")
plt.savefig("vanilla.png")

In [25]:
df = pd.DataFrame(history_callback_dropout.history)
df[['acc', 'val_acc']].plot()
plt.ylabel("accuracy")
df[['loss', 'val_loss']].plot(linestyle='--', ax=plt.twinx())
plt.ylabel("loss")
plt.savefig("dropout.png")

In [26]:
model_vanilla.summary()
model_dropout.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_44 (Dense)             (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_97 (Dense)             (None, 256)               200960    
_________________________________________________________________
dropout_27 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_98 (Dense)             (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 

In [27]:
##### evaluation and compare #####
score_vanilla = model_vanilla.evaluate(X_test, y_test, verbose=0)
print("------- Vanilla Model -------")
print("Test loss: {:.3f}".format(score_vanilla[0]))
print("Test Accuracy: {:.3f}".format(score_vanilla[1]))

score_dropout = model_dropout.evaluate(X_test, y_test, verbose=0)
print("------- Dropout Model -------")
print("Test loss: {:.3f}".format(score_dropout[0]))
print("Test Accuracy: {:.3f}".format(score_dropout[1]))

------- Vanilla Model -------
Test loss: 0.109
Test Accuracy: 0.976
------- Dropout Model -------
Test loss: 0.079
Test Accuracy: 0.984
